# Načítání ze souboru

## Základní operace

In [ ]:
file = open("../../examples/config.ini", "r")

data = file.read()

print(data)

file.close()

soubor lze otevřít v několika modech:
- 'r' pro čtení
- 'w' pro zápis
- 'a' pro zápis v režimu append - zápis na konec

ke každému lze přidat ještě modifier 'b' pro práci v binárním režimu

Soubor je po dokončení práce třeba vždy uzavřít. Pokud soubor ponecháme otevřený, může dojít k nečekaným výsledkům. V příkladu níže vidíme, že při načtení ze souboru pomocí handle `f3` nevidíme změny zapsané pomocí handle `f2`.

In [ ]:
from os import remove

filename = "tmpfile.txt"

f1 = open(filename,'w')
f1.write("content")
f1.close()

f2 = open(filename, 'a')
f2.write(' - incorrectly modified')

f3 = open(filename,'r')
print(f3.read())

f2.close()
f3.close()

remove(filename)

Zatímco pokud soubor vždy pečlivě uzavřeme, změny se zapíšou.

In [ ]:
from os import remove

filename = "tmpfile.txt"

f1 = open(filename,'w')
f1.write("content")
f1.close()

f2 = open(filename, 'a')
f2.write(' - correctly modified')
f2.close()

f3 = open(filename,'r')
print(f3.read())
f3.close()

remove(filename)

- dobrým zvykem je, zavírat otevřený soubor, jakmile ho dále nepotřebujeme
- jakmile napíšete `open`, napište rovnou i `close`

## `with` statement a context manager

- jednou možností, jak bezpečněji otevírat soubory, je `with` statement.
- veškerá práce se odehrává v odsazeném bloku a po jeho skončení se soubor uzavře
- https://peps.python.org/pep-0343/

In [ ]:
with open("../../examples/config.ini", "r") as f:
    data = f.read()

for line in data.splitlines():
    print(line)

`with` statement je ve skutečnosti operuje s něčím, co se nazývá *context manager protocol*. Context manager protocol naplňuje každý objekt, který má definované metody `__enter__` a `__exit__`

Uzavírání zdrojů po dokončení práce se tradičně řeší pomocí bloku `finally`. A skutečně, řádky

```python
with EXPR as var:
    BLOCK
```

jsou zhruba ekvivalentní řádkům

```python
VAR = EXPR
VAR.__enter__()
try:
    BLOCK
finally:
    VAR.__exit__()
```

Předchozí příklad lze přepsat jako

```python
f = open("../examples/config.ini", "r")
f.__enter__()
try:
    data = f.read()
finally:
    f.__exit__()
```

In [ ]:
f = open("../../examples/config.ini", "r")
f.__enter__()
try:
    data = f.read()
finally:
    f.__exit__()
print(data)

Přidejme si typický příklad - práce s databízí:

In [ ]:
class DatabaseConnection:
    def __init__(self, db_name):
        self.db_name = db_name

    def __enter__(self):
        print(f"connecting to database {self.db_name}")
        return self
    
    def execute(self, query):
        print(f"executing query: {query}")
        
    def execute_with_exception(self, query):
        raise Exception(f"a generic exception for this example")

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f"Exception {exc_type} caught, rollin back changes")
        else:
            print("committing changes")
        print("closing connection")


V případě, že se nic nepokazí, je průběh přímočarý:

In [ ]:
with DatabaseConnection("my_db.sqlite") as db:
    db.execute("CREATE TABLE IF NOT EXISTS my_table (id INTEGER PRIMARY KEY, name TEXT);")
    db.execute("INSERT INTO my_table (name) VALUES ('Alice');")

Pokud ale dojde k výjimce (zde reprezentováno metodou `execute_with_exception`), připojení k databázi se normálně ukončí i přes to, že výjimka "probublá" ven.

In [ ]:
try:
    with DatabaseConnection("my_db.sqlite") as db:
        db.execute("CREATE TABLE IF NOT EXISTS my_table (id INTEGER PRIMARY KEY, name TEXT);")
        db.execute_with_exception("INSERT INTO my_table (name) VALUES ('Alice');")
except Exception:
    print("database query failed but database connection was safely closed")

## Načítání známých datových typů

Následuje orientační přehled formátů, které lze číst pomocí Python Standard Library - tj. není nutné nic instalovat. Seznam není vyčerpávající.

### `ini`

Formát `ini` je celkem známý, prostý formát konfiguračních souborů, příklad z Wiki:

```ini
; last modified 1 April 2001 by John Doe
[owner]
name = John Doe
organization = Acme Widgets Inc.

[database]
; use IP address in case network name resolution is not working
server = 192.0.2.62     
port = 143
file = "payroll.dat"
```

Problém s ini formátem je ten, že není příliš standardizování. Některé varianty nedovolují komentáře, jiné nedovolují sekce.

V Python je k dispozici balík `configparser`, který nějaké `ini`-like soubory číst umí.

Nevýhody:
- nepříjemná práce
- neidentifikuje datové typy hodnot
- vytváří vlastní (sice iterovatelné) typy, ačkoliv by byl lepší slovník

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read("../../examples/config.ini")

for section in config.sections():
    for key, val in config[section].items():
        print(key, val, type(val))

Existuje alternativa - [toml](https://toml.io/en/) - Tom's Obvious Minimal Language

- podobný ini, ale s konkrétní specifikací
- široká podpora napříč jazyky
- rozeznává datové typy hodnot
- vše dává k dispozici jako slovník

V Pythonu dostupný jako externí balík `tomli`, od verze 3.11 dokonce součástí Pythonu jako `tomllib`

In [ ]:
import tomli

with open("../../examples/config.toml", "rb") as file:
    cfg = tomli.load(file)

print(cfg)

### `json`

JSON - JavaScript Object Notation

specifikace [json.org](https://www.json.org/json-en.html)

- v pythonu s pomocí knihovny [json](https://docs.python.org/3/library/json.html)
- formát JSON je mimořádně rozšířen
- knihovna snadná na používání
- má zabudovaný "pretty print"

In [ ]:
import json

with open("../../examples/config.json", "r") as file:
    cfg = json.load(file)
    
for key, val in cfg["section a"].items():
    print(key, val, type(val))
    
print(json.dumps(cfg, indent=4))

### `csv` - comma separated values

Další velmi rozšířený formát s nejasnou specifikací. Python nabízí ve standard library modul pro práci s `csv`: [csv](https://docs.python.org/3/library/csv.html).

```{admonition} Osobní názor
:class: important
S modulem `csv` se pracuje špatně. Je divný a omezený.
```

Trochu robustnější možnosti čtení `csv` souborů nabízí moduly jako `numpy` nebo `pandas`.